<a href="https://colab.research.google.com/github/hargurjeet/Maersk/blob/main/Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q gitpython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.3 MB/s eta 0:00:00


In [17]:
## Python Imports
import pandas as pd
import numpy as np
from git import Repo
import os
import matplotlib.pyplot as plt
import seaborn as sns

## Libraries for model building
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV

In [3]:
# Clone the GitHub repository
repo_url = 'https://github.com/hargurjeet/Maersk'
repo_dir = 'Maersk'
Repo.clone_from(repo_url, repo_dir)

<git.repo.base.Repo '/content/Maersk/.git'>

In [4]:
import pandas as pd

file_path = "/content/Maersk/vessel_data.csv"
date_cols = ['arrival_date', 'departure_date']

dtypes = {
    'imo': 'object',
}

vessel_data = pd.read_csv(file_path, parse_dates=date_cols, dtype=dtypes,)
pd.set_option('display.max_columns', None)
vessel_data.head(2)


,Unnamed: 0,call_id,arrival_date,departure_date,arrival_time,departure_time,imo,vessel_name_ais,vessel_type_ais,Country,CountryCode,Port,Terminal,Duration,Service_name,length,prev_departure_time,vesteu,prev_port,prev_arrival_time,prev_duration,prev_leg_duration,prev_leg_distance_nm,prev_leg_stationary_hours,prev_call_id,next_port,next_arrival_time,next_duration,next_leg_duration,next_leg_distance_nm,next_leg_stationary_hours,next_call_id,total_duration_shipment,total_duration_shipment_days,total_distance_shipment
0,0,7b75a18f54d78c79df4e9ceb29c65c60d6b5c7ae,2022-01-05,2022-01-07,2022-01-05 23:00:19+00:00,2022-01-07 21:04:26+00:00,9525376,SAFMARINE CHAMBAL,Container Ship,China,CN,Xiamen,Xiamen Songyu Container Terminal,46.068611,ML - TPX,249.12,2021-12-14 07:00:05+00:00,4496,Haiphong,2021-12-13 17:02:55+00:00,13.952778,544.003889,1598.620103,28,705238686bb73f1947352631d015ad9485d78a9c,Los Angeles,2022-02-10 12:01:14+00:00,207.020556,806.946667,7407.903024,201.0,c77797c9b9822d7a799e6f7664f7d6be5f73f270,1846.992500,76.958021,9006.523126
1,1,c77797c9b9822d7a799e6f7664f7d6be5f73f270,2022-02-10,2022-02-19,2022-02-10 12:01:14+00:00,2022-02-19 03:02:28+00:00,9525376,SAFMARINE CHAMBAL,Container Ship,United States,US,Los Angeles,Apmt Pier 400,207.020556,ML - TPX,249.12,2022-01-07 21:04:26+00:00,4496,Xiamen,2022-01-05 23:00:19+00:00,46.068611,806.946667,7407.903024,201,7b75a18f54d78c79df4e9ceb29c65c60d6b5c7ae,Qingdao,2022-03-08 22:27:41+00:00,105.949167,427.420278,5967.080241,25.0,0dccb61f51069f40de6592464459777a2eb73334,1819.405278,75.808553,13374.983265


In [5]:
#fixing datatypes of columns
def remove_timezone_info(data):
    return data.str[:-6]

def convert_to_datetime(data):
    return pd.to_datetime(data, format="%Y-%m-%d %H:%M:%S", errors='coerce')

columns = ['arrival_time', 'departure_time', 'prev_arrival_time', 'prev_departure_time', 'next_arrival_time']

for col in columns:
    vessel_data[col] = remove_timezone_info(vessel_data[col])
    vessel_data[col] = convert_to_datetime(vessel_data[col])


In [6]:
vessel_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494042 entries, 0 to 494041
Data columns (total 35 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   Unnamed: 0                    494042 non-null  int64         
 1   call_id                       494042 non-null  object        
 2   arrival_date                  494042 non-null  datetime64[ns]
 3   departure_date                494042 non-null  datetime64[ns]
 4   arrival_time                  493604 non-null  datetime64[ns]
 5   departure_time                493614 non-null  datetime64[ns]
 6   imo                           494042 non-null  object        
 7   vessel_name_ais               494042 non-null  object        
 8   vessel_type_ais               494039 non-null  object        
 9   Country                       494042 non-null  object        
 10  CountryCode                   493669 non-null  object        
 11  Port         

In [7]:
vessel_data.head(2)

,Unnamed: 0,call_id,arrival_date,departure_date,arrival_time,departure_time,imo,vessel_name_ais,vessel_type_ais,Country,CountryCode,Port,Terminal,Duration,Service_name,length,prev_departure_time,vesteu,prev_port,prev_arrival_time,prev_duration,prev_leg_duration,prev_leg_distance_nm,prev_leg_stationary_hours,prev_call_id,next_port,next_arrival_time,next_duration,next_leg_duration,next_leg_distance_nm,next_leg_stationary_hours,next_call_id,total_duration_shipment,total_duration_shipment_days,total_distance_shipment
0,0,7b75a18f54d78c79df4e9ceb29c65c60d6b5c7ae,2022-01-05,2022-01-07,2022-01-05 23:00:19,2022-01-07 21:04:26,9525376,SAFMARINE CHAMBAL,Container Ship,China,CN,Xiamen,Xiamen Songyu Container Terminal,46.068611,ML - TPX,249.12,2021-12-14 07:00:05,4496,Haiphong,2021-12-13 17:02:55,13.952778,544.003889,1598.620103,28,705238686bb73f1947352631d015ad9485d78a9c,Los Angeles,2022-02-10 12:01:14,207.020556,806.946667,7407.903024,201.0,c77797c9b9822d7a799e6f7664f7d6be5f73f270,1846.992500,76.958021,9006.523126
1,1,c77797c9b9822d7a799e6f7664f7d6be5f73f270,2022-02-10,2022-02-19,2022-02-10 12:01:14,2022-02-19 03:02:28,9525376,SAFMARINE CHAMBAL,Container Ship,United States,US,Los Angeles,Apmt Pier 400,207.020556,ML - TPX,249.12,2022-01-07 21:04:26,4496,Xiamen,2022-01-05 23:00:19,46.068611,806.946667,7407.903024,201,7b75a18f54d78c79df4e9ceb29c65c60d6b5c7ae,Qingdao,2022-03-08 22:27:41,105.949167,427.420278,5967.080241,25.0,0dccb61f51069f40de6592464459777a2eb73334,1819.405278,75.808553,13374.983265


# Feature Engineering

In [9]:
## Splitting data and time
def part_of_dates(df, column):
  """ Extrace Year, Month, day, weekday and hour from a date value """
  df[column+ '_year']= df[column].dt.year
  df[column+ '_month']= df[column].dt.month
  df[column+ '_day']= df[column].dt.day
  df[column+ '_week']= df[column].dt.weekday
  df[column+ '_hour']= df[column].dt.hour

part_of_dates(vessel_data, 'arrival_date')
part_of_dates(vessel_data, 'departure_date')
part_of_dates(vessel_data, 'arrival_time')
part_of_dates(vessel_data, 'departure_time')
part_of_dates(vessel_data, 'prev_departure_time')
part_of_dates(vessel_data, 'prev_arrival_time')
part_of_dates(vessel_data, 'next_arrival_time')

In [11]:
vessel_data.head(2)

,Unnamed: 0,call_id,arrival_date,departure_date,arrival_time,departure_time,imo,vessel_name_ais,vessel_type_ais,Country,CountryCode,Port,Terminal,Duration,Service_name,length,prev_departure_time,vesteu,prev_port,prev_arrival_time,prev_duration,prev_leg_duration,prev_leg_distance_nm,prev_leg_stationary_hours,prev_call_id,next_port,next_arrival_time,next_duration,next_leg_duration,next_leg_distance_nm,next_leg_stationary_hours,next_call_id,total_duration_shipment,total_duration_shipment_days,total_distance_shipment,arrival_date_year,arrival_date_month,arrival_date_day,arrival_date_week,arrival_date_hour,departure_date_year,departure_date_month,departure_date_day,departure_date_week,departure_date_hour,arrival_time_year,arrival_time_month,arrival_time_day,arrival_time_week,arrival_time_hour,departure_time_year,departure_time_month,departure_time_day,departure_time_week,departure_time_hour,prev_departure_time_year,prev_departure_time_month,prev_departure_time_day,prev_departure_time_week,prev_departure_time_hour,prev_arrival_time_year,prev_arrival_time_month,prev_arrival_time_day,prev_arrival_time_week,prev_arrival_time_hour,next_arrival_time_year,next_arrival_time_month,next_arrival_time_day,next_arrival_time_week,next_arrival_time_hour
0,0,7b75a18f54d78c79df4e9ceb29c65c60d6b5c7ae,2022-01-05,2022-01-07,2022-01-05 23:00:19,2022-01-07 21:04:26,9525376,SAFMARINE CHAMBAL,Container Ship,China,CN,Xiamen,Xiamen Songyu Container Terminal,46.068611,ML - TPX,249.12,2021-12-14 07:00:05,4496,Haiphong,2021-12-13 17:02:55,13.952778,544.003889,1598.620103,28,705238686bb73f1947352631d015ad9485d78a9c,Los Angeles,2022-02-10 12:01:14,207.020556,806.946667,7407.903024,201.0,c77797c9b9822d7a799e6f7664f7d6be5f73f270,1846.992500,76.958021,9006.523126,2022,1,5,2,0,2022,1,7,4,0,2022.0,1.0,5.0,2.0,23.0,2022.0,1.0,7.0,4.0,21.0,2021.0,12.0,14.0,1.0,7.0,2021.0,12.0,13.0,0.0,17.0,2022.0,2.0,10.0,3.0,12.0
1,1,c77797c9b9822d7a799e6f7664f7d6be5f73f270,2022-02-10,2022-02-19,2022-02-10 12:01:14,2022-02-19 03:02:28,9525376,SAFMARINE CHAMBAL,Container Ship,United States,US,Los Angeles,Apmt Pier 400,207.020556,ML - TPX,249.12,2022-01-07 21:04:26,4496,Xiamen,2022-01-05 23:00:19,46.068611,806.946667,7407.903024,201,7b75a18f54d78c79df4e9ceb29c65c60d6b5c7ae,Qingdao,2022-03-08 22:27:41,105.949167,427.420278,5967.080241,25.0,0dccb61f51069f40de6592464459777a2eb73334,1819.405278,75.808553,13374.983265,2022,2,10,3,0,2022,2,19,5,0,2022.0,2.0,10.0,3.0,12.0,2022.0,2.0,19.0,5.0,3.0,2022.0,1.0,7.0,4.0,21.0,2022.0,1.0,5.0,2.0,23.0,2022.0,3.0,8.0,1.0,22.0


# Training a baseline model

In [12]:
vessel_data.columns

Index(['Unnamed: 0', 'call_id', 'arrival_date', 'departure_date',
       'arrival_time', 'departure_time', 'imo', 'vessel_name_ais',
       'vessel_type_ais', 'Country', 'CountryCode', 'Port', 'Terminal',
       'Duration', 'Service_name', 'length', 'prev_departure_time', 'vesteu',
       'prev_port', 'prev_arrival_time', 'prev_duration', 'prev_leg_duration',
       'prev_leg_distance_nm', 'prev_leg_stationary_hours', 'prev_call_id',
       'next_port', 'next_arrival_time', 'next_duration', 'next_leg_duration',
       'next_leg_distance_nm', 'next_leg_stationary_hours', 'next_call_id',
       'total_duration_shipment', 'total_duration_shipment_days',
       'total_distance_shipment', 'arrival_date_year', 'arrival_date_month',
       'arrival_date_day', 'arrival_date_week', 'arrival_date_hour',
       'departure_date_year', 'departure_date_month', 'departure_date_day',
       'departure_date_week', 'departure_date_hour', 'arrival_time_year',
       'arrival_time_month', 'arrival_time_da

In [15]:
# Split data into features and target
rel_cols = ['vessel_type_ais', 'Country', 'Port','Duration','length', 'prev_departure_time', 'vesteu','prev_port','prev_duration', 'prev_leg_duration',\
            'prev_leg_distance_nm', 'prev_leg_stationary_hours','next_port','next_duration', 'next_leg_duration', 'next_leg_distance_nm',\
            'next_leg_stationary_hours','total_duration_shipment', 'total_distance_shipment', 'arrival_date_year', 'arrival_date_month',\
            'arrival_date_day', 'arrival_date_week', 'arrival_date_hour', 'departure_date_year', 'departure_date_month', 'departure_date_day',\
            'departure_date_week', 'departure_date_hour', 'arrival_time_year', 'arrival_time_month', 'arrival_time_day', 'arrival_time_week',\
            'arrival_time_hour', 'departure_time_year', 'departure_time_month', 'departure_time_day', 'departure_time_week', 'departure_time_hour',\
            'prev_departure_time_year', 'prev_departure_time_month', 'prev_departure_time_day', 'prev_departure_time_week', 'prev_departure_time_hour', \
            'prev_arrival_time_year', 'prev_arrival_time_month', 'prev_arrival_time_day', 'prev_arrival_time_week', 'prev_arrival_time_hour',\
            'next_arrival_time_year', 'next_arrival_time_month', 'next_arrival_time_day', 'next_arrival_time_week', 'next_arrival_time_hour']

# Split data into features and target
X = vessel_data[rel_cols].drop('prev_leg_stationary_hours',axis=1)
y = vessel_data['prev_leg_stationary_hours']

print(vessel_data.shape)

(494042, 70)


In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define preprocessing steps for numerical and categorical columns
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ])